In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("loan_approval_data.csv")
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'loan_approval_data.csv'

1. handling missing values

In [4]:
#1. handling missing values

#numerical variables
#to fill values usig library
from sklearn.impute import SimpleImputer
num_imp = SimpleImputer(strategy="mean")
#fill age with mean of age and etc
numerical_cols = df.select_dtypes(include=["float64", "int64"]).columns
df[numerical_cols] = num_imp.fit_transform(df[numerical_cols])

#cartegorical variables (based on mode)
categorical_cols = df.select_dtypes(include=["object"]).columns
cat_imp = SimpleImputer(strategy="most_frequent")
df[categorical_cols] = cat_imp.fit_transform(df[categorical_cols])

#removing unecessary feature
#Remove Applicant ID
df = df.drop("Applicant_ID", axis=1)

NameError: name 'df' is not defined

2. EDA

#2. EDA
#how balanced our classes are?
classes_count = df["Loan_Approved"].value_counts()
plt.pie(classes_count, labels=["No", "Yes"], autopct="%1.1f%%")
plt.title("Is Loan approved or not?")


In [ ]:
#analyse categories
gender_cnt= df["Gender"].value_counts()
ax = sns.barplot(gender_cnt)
ax.bar_label(ax.containers[0])

In [ ]:
df.head()

In [ ]:
#analyse income -logically thinking
sns.histplot(
    data = df,
    x = "Applicant_Income",
    bins =20
)

In [ ]:
sns.histplot(
    data = df,
    x = "Coapplicant_Income",
    bins =20
)

In [ ]:
#detecing outliers
fig, axes = plt.subplots(2,2)
sns.boxplot(ax=axes[0,0], data=df, x="Loan_Approved", y="Applicant_Income")
sns.boxplot(ax=axes[0,1], data=df, x="Loan_Approved", y="Credit_Score")
sns.boxplot(ax=axes[1,0], data=df, x="Loan_Approved", y="DTI_Ratio")
sns.boxplot(ax=axes[1,1], data=df, x="Loan_Approved", y="Savings")

plt.tight_layout()

In [ ]:
#connecting between features with output
#credit score wth loan approved
sns.histplot(
    data=df,
    x="Credit_Score",
    hue="Loan_Approved",
    bins=20,
    multiple="dodge"
)



3. Feature Encoding

#enconding categroical variables
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
#label encoder
le = LabelEncoder()
df["Education_Level"] = le.fit_transform(df["Education_Level"])
df["Loan_Approved"] = le.fit_transform(df["Loan_Approved"])

#one hot encoder
cols = ["Employment_Status", "Marital_Status", "Loan_Purpose",
        "Property_Area", "Gender", "Employer_Category"]

ohe = OneHotEncoder(drop="first", sparse_output=False, handle_unknown="ignore")
encoded = ohe.fit_transform(df[cols])

encoded_df = pd.DataFrame(encoded, columns=ohe.get_feature_names_out(cols), index=df.index)

df = pd.concat([df.drop(columns=cols), encoded_df],axis=1)


In [ ]:
df.info()

# Correlation Heatmap

nums_cols = df.select_dtypes(include = "number")
corr_matrix = nums_cols.corr()

In [ ]:
#would like to see the relatonship with the target 
nums_cols.corr()["Loan_Approved"].sort_values(ascending=False)

In [ ]:
#we show the above using heat map 
plt.figure(figsize=(15,8))
sns.heatmap(
    corr_matrix,
    annot=True,
    fmt=".2f",
    cmap="coolwarm"
)
    

# Train-Test-SPlit + feature scaling


X = df.drop("Loan_Approved", axis=1)
y = df["Loan_Approved"]


In [ ]:
y.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_test.head()

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
X_train_scaled

Train & Evaluate Models

#the problems statments says we need to:
#1. reduce high-risk customers that gets approved therefore we trying to minimise false positive (FP)
#(we use the parameter Precision for this)
#2. good customers that gets rejected therefore we trying to minimise False negative (FN)
#(we used the parameter Recall for this)
#thus precision is important then it will be recall and then f1-score (its used to keep balance)
#logistic Regressoion 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix, recall_score, f1_score

log_model = LogisticRegression()
log_model.fit(X_train_scaled, y_train)
y_pred = log_model.predict(X_test_scaled)

#evaluation 
print("logistic Regression Model")
print("Precision: ", precision_score(y_test, y_pred))
print("Recall: ", recall_score(y_test, y_pred))
print("F1_score: ", f1_score(y_test, y_pred))
print("Accuracy: ", accuracy_score(y_test, y_pred))
print("CM: ", confusion_matrix(y_test, y_pred))
      

In [ ]:
#KNN  
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train_scaled, y_train)
y_pred = knn_model.predict(X_test_scaled)

#evaluation 
print("KNN Model")
print("Precision: ", precision_score(y_test, y_pred))
print("Recall: ", recall_score(y_test, y_pred))
print("F1_score: ", f1_score(y_test, y_pred))
print("Accuracy: ", accuracy_score(y_test, y_pred))
print("CM: ", confusion_matrix(y_test, y_pred))
      

In [ ]:
#Naive Bayes
from sklearn.naive_bayes import GaussianNB

nb_model = GaussianNB()
nb_model.fit(X_train_scaled, y_train)
y_pred = nb_model.predict(X_test_scaled)

#evaluation 
print("Naive Bayes Model")
print("Precision: ", precision_score(y_test, y_pred))
print("Recall: ", recall_score(y_test, y_pred))
print("F1_score: ", f1_score(y_test, y_pred))
print("Accuracy: ", accuracy_score(y_test, y_pred))
print("CM: ", confusion_matrix(y_test, y_pred))
      

Best model on the basis of precision => Naive Bayes

# Feature Engineering

In [ ]:
#CREDIT-SCORE DTI-RATION STOOD OUT BASED ON HEATMAP
#we can increase the impract of these features

#converting these features into polynomial features
df["DTI_Ratio_sq"] = df["DTI_Ratio"] ** 2
df["Credit_Score_sq"]  = df["Credit_Score"] ** 2


X = df.drop(columns=["Loan_Approved", "Credit_Score", "DTI_Ratio"])
y= df["Loan_Approved"]

#train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
#logistic Regressoion 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix, recall_score, f1_score

log_model = LogisticRegression()
log_model.fit(X_train_scaled, y_train)
y_pred = log_model.predict(X_test_scaled)

#evaluation 
print("logistic Regression Model")
print("Precision: ", precision_score(y_test, y_pred))
print("Recall: ", recall_score(y_test, y_pred))
print("F1_score: ", f1_score(y_test, y_pred))
print("Accuracy: ", accuracy_score(y_test, y_pred))
print("CM: ", confusion_matrix(y_test, y_pred))

In [ ]:
#KNN  
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train_scaled, y_train)
y_pred = knn_model.predict(X_test_scaled)

#evaluation 
print("KNN Model")
print("Precision: ", precision_score(y_test, y_pred))
print("Recall: ", recall_score(y_test, y_pred))
print("F1_score: ", f1_score(y_test, y_pred))
print("Accuracy: ", accuracy_score(y_test, y_pred))
print("CM: ", confusion_matrix(y_test, y_pred))

In [ ]:
#Naive Bayes
from sklearn.naive_bayes import GaussianNB

nb_model = GaussianNB()
nb_model.fit(X_train_scaled, y_train)
y_pred = nb_model.predict(X_test_scaled)

#evaluation 
print("Naive Bayes Model")
print("Precision: ", precision_score(y_test, y_pred))
print("Recall: ", recall_score(y_test, y_pred))
print("F1_score: ", f1_score(y_test, y_pred))
print("Accuracy: ", accuracy_score(y_test, y_pred))
print("CM: ", confusion_matrix(y_test, y_pred))

# Naive Bayes is still the best model in terms of precision. logistic regression is the second best (improved after feature engineering - to keep balance (F1-score) then choose logistic regression))